##Random Forest seems to perform better (overall accuracy) than Gradient Boosting

In [1]:
pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 325.1 kB/s eta 0:05:07
   ---------------------------------------- 0.1/99.8 MB 1.0 MB/s eta 0:01:38
   ---------------------------------------- 0.4/99.8 MB 2.5 MB/s eta 0:00:40
   ---------------------------------------- 0.7/99.8 MB 3.7 MB/s eta 0:00:27
   ---------------------------------------- 1.1/99.8 MB 4.3 MB/s eta 0:00:23
    --------------------------------------- 1.4/99.8 MB 5.1 MB/s eta 0:00:20
    --------------------------------------- 1.7/99.8 MB 5.0 MB/s eta 0:00:20
    --------------------------------------- 1.9/99.8 MB 4.9 MB/s eta 0:00:20
    ---------------------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

behav = pd.read_csv('../data/300behavior.csv')

# Assuming behav is your DataFrame and it's already imported
# Replace all occurrences of 3 with 2 in the 'NextLever' column
behav['NextLever'] = behav['NextLever'].replace(3, 2)
B23behav = behav[(behav['Trial#'] > 20) & ~(behav['Trial#'].isin([40, 60]))]

B23behav.head

<bound method NDFrame.head of       ID    Date  Trial#  Lever  NextLever  WSLS  LeverZmax  LeverZmin  \
20    16  230123      21      3          2     1   2.401602  -1.003824   
21    16  230123      22      3          2     1   2.926855  -1.217437   
22    16  230123      23      3          2     1   1.696856  -1.406294   
23    16  230123      24      3          2     1   1.724995  -1.203465   
24    16  230123      25      3          2     1   0.747431  -1.450247   
...   ..     ...     ...    ...        ...   ...        ...        ...   
1672  32  230407      51      3          1     3   3.332265  -1.602789   
1673  32  230407      52      1          0     0   4.506867  -2.054242   
1674  32  230407      55      1          2     0   0.637471  -3.201188   
1675  32  230407      56      2          0     2   2.911747  -0.353442   
1676  32  230407      59      1          0     0   1.568098  -1.403199   

        LeverAUC    HLZmax    HLZmin       HLAUC  
20     80.581480  2.714354 -3.

In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Example dataset
# Replace this with your actual dataset
df = B23behav

# Assuming df is your DataFrame and 'NextLever' is the target variable
X = B23behav[['LeverZmax','LeverZmin','HLZmin','HLZmax']]  # Predictor variables
y = B23behav['NextLever']  # Target variable with three categories

# Splitting the dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Applying SMOTE (Synthetic Minority Over-sampling Technique)
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [4]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

# Convert the dataset into an optimized data structure called Dmatrix
dtrain = xgb.DMatrix(X_train_res, label=y_train_res)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up the parameters for XGBoost
params = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 3
}

# Train the model
bst = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
y_pred = bst.predict(dtest)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.05      0.21      0.08        19
           1       0.12      0.20      0.15        30
           2       0.83      0.57      0.68       263

    accuracy                           0.52       312
   macro avg       0.33      0.33      0.30       312
weighted avg       0.71      0.52      0.59       312

Confusion Matrix:
[[  4   2  13]
 [  5   6  19]
 [ 69  43 151]]


In [7]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

# Convert the dataset into an optimized data structure called Dmatrix
dtrain = xgb.DMatrix(X_train_res, label=y_train_res)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up the parameters for XGBoost
params = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 3
}

# Train the model
bst = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
y_pred = bst.predict(dtest)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.03      0.05      0.03        19
           1       0.25      0.43      0.32        30
           2       0.88      0.75      0.81       263

    accuracy                           0.67       312
   macro avg       0.39      0.41      0.39       312
weighted avg       0.77      0.67      0.71       312

Confusion Matrix:
[[  1   3  15]
 [  6  13  11]
 [ 32  35 196]]
